# Continuous Usage Discovery

Goal: Quick script that can find continuous usage of a resource in a given time period.

#### Parameters

In [1]:
customerName = 'Honeywell'

# bounding dates
startDate = '2023-11-01'
endDate = '2024-05-01'

# analysis parameters
params = {
    "Lighting": {"percent_of_peak": 0.5, "min_length_hours": 48}, 
    "HVAC": {"percent_of_peak": 0.5, "min_length_hours": 48},
    "Other": {"percent_of_peak": 0.5, "min_length_hours": 48}
}

### Imports and Initializations

In [2]:
import pandas as pd
import datetime as dt

from redaptipy.db_clients import rdp_db_client
from redaptipy.api_clients import rdp_partner_api_client

from plotly import graph_objects as go

rdp_client = rdp_db_client.RdpDBClient()
rdp_api_client = rdp_partner_api_client.RdpApiClient()

### Asset Selection

In [3]:
q = """SELECT c.id, 
              c.description,
              p.name as panel_name,
              m.verified_date,
              bs.name as building_system,
              s.address1,
                s.city,
                s.state,
                s.timezone
 FROM circuit c
JOIN building_system bs ON c.building_system_id = bs.id
JOIN panel p ON c.panel_id = p.id
JOIN meter m ON c.meter_id = m.id
JOIN site s ON c.site_id = s.id
JOIN customer cu ON s.customer_id = cu.id
WHERE cu.name = 'Honeywell'
and bs.name not in ('Other', 'Other Submetered', 'Panel Feed', 'Main Panel', 'Main Feed', 'Transformer')
and (c.description not ilike '%exit%' 
    and c.description not ilike '%emergency%' 
    and c.description not ilike '%egress%' 
    and c.description not ilike '%security%')
and c.description is not null"""


In [4]:
circuits = rdp_client.query(q).set_index('id')

### Data Download

In [5]:
df = rdp_api_client.get_data_by_circuit(
    circuit_ids=circuits.index.to_list(), 
    start_date=dt.datetime.strptime(startDate, '%Y-%m-%d'),
    end_date=dt.datetime.strptime(endDate, '%Y-%m-%d'),
    resolution=15,
    power_parameter='activePower'
    )

df.to_csv('co_data.csv')

### Analysis

In [29]:
equipment_df = df.groupby([circuits['address1'], 
                           circuits['description'],
                           circuits['building_system'], 
                           circuits['timezone'],
                           circuits['verified_date']], axis=1).sum()

In [35]:
blocks_df = pd.DataFrame()

for c in equipment_df.columns:
    site, equipment_name, system, timezone, metering_start = c[0], c[1], c[2], c[3], c[4]
    system_params = params.get(system, params['Other'])
    percent_of_peak = system_params['percent_of_peak']
    min_length_hours = system_params['min_length_hours']    

    equipment = equipment_df[c][metering_start:]
    equipment = equipment.tz_localize('UTC').tz_convert(timezone).tz_localize(None)
    
    
    thresholded = equipment_df[c] / equipment_df[c].quantile(.99) > percent_of_peak
    blocks = thresholded.diff().abs().cumsum()
    matches = blocks.mask(~thresholded, 0)
    
    blocks = pd.DataFrame(data={'length':matches.groupby(matches).count() * 15,
                                'start':matches.groupby(matches).apply(lambda s: s.index[0]),
                                'max': equipment_df[c].max()})
    blocks = blocks[blocks.length > (min_length_hours*60)]
    blocks = blocks.drop(0, errors='ignore')

    fig = go.Figure()

    figStart = pd.to_datetime('2025-01-01')
    figEnd = pd.to_datetime('2020-01-01')

    print(f'{site} - {equipment_name} - {system}')
    print(blocks.shape[0])
    if blocks.shape[0] > 0:
        for i, block in blocks.iterrows():
            if block['max'] < 1: continue
            block_info = {'site': site,
                          'equipment': equipment_name,
                          'building_system': system,
                          'length': block['length'],
                          'start': block['start'],
                          'end': block['start'] + pd.Timedelta(seconds=60*block['length']),
                          'verified_date': metering_start
                          }

            insert_index = len(blocks_df)
            for k, v in block_info.items():
                blocks_df.loc[insert_index, k] = v
                blocks_df.loc[insert_index, 'max_usage_in_block'] = equipment_df[block_info['start']:block_info['end']][c].max()
                blocks_df.loc[insert_index, 'avg_usage_in_block'] = equipment_df[block_info['start']:block_info['end']][c].mean()
                blocks_df.loc[insert_index, 'total_usage_in_block'] = equipment_df[block_info['start']:block_info['end']][c].sum()/4
                blocks_df.loc[insert_index, 'avg_usage'] = equipment_df[c].mean()
                blocks_df.loc[insert_index, 'max_usage'] = equipment_df[c].max()
                blocks_df.loc[insert_index, 'total_usage'] = equipment_df[c].sum()/4
                blocks_df.loc[insert_index, 'block_percentage_of_use'] = blocks_df.loc[insert_index, 'total_usage_in_block'] / blocks_df.loc[insert_index, 'total_usage']

            figStart = min(figStart, block_info['start'])
            figEnd = max(figEnd, block_info['start'] + pd.Timedelta(seconds=60*block_info['length']))

            fig.add_shape(type='rect',
                        x0=block_info['start'],
                        y0=0,
                        x1=block_info['end'],
                        y1=equipment_df[block_info['start']:block_info['end']][c].max(),
                        line=dict(width=0),
                        fillcolor='red',
                        opacity=0.5)

        fig.add_scatter(x=matches[figStart:figEnd].index, y=equipment_df[figStart:figEnd][c])

        fig.update_layout(template='simple_white')
        fig.update_xaxes(title='Time')
        fig.update_yaxes(title='Power (kW)')
        
        fig.write_html(f'Honeywell/continuousUsageCharts/{site}_{equipment_name.replace("/","").strip()}.html')

 Fire-Lite Place -  Variable Air volume 5 - PLP2 - HVAC
0
 Fire-Lite Place - 11 High Bay Lighting - HP-G15 - Lighting
4
 Fire-Lite Place - 24 Volts Output by Red Stripped Battery Chargers - HVP-1 - Plug Load
1
 Fire-Lite Place - 480v Twist Lock Column Outlet - HVP-1 - Distribution Center Area - Plug Load
0
 Fire-Lite Place - 9 High Bay Lighting - HP-G15 - Lighting
12
 Fire-Lite Place - Air Compressor - HP-N17 - HVAC
1
 Fire-Lite Place - Air Handling Unit Rooftop #40 Office - HVAC - HVAC
0
 Fire-Lite Place - Air Handling Unit Rooftop #42 Office - HVAC - HVAC
1
 Fire-Lite Place - Air Handling Unit Rooftop High Bay #31 - HVAC - HVAC
0
 Fire-Lite Place - Air Handling Unit Rooftop High Bay #33 - HVAC - HVAC
5
 Fire-Lite Place - Air Handling Unit Rooftop High Bay #34 - HVAC - HVAC
1
 Fire-Lite Place - Air Handling Unit Rooftop Low Bay #35 - HVAC - HVAC
0
 Fire-Lite Place - Air Handling Unit Rooftop Low Bay #37 - HVAC - HVAC
0
 Fire-Lite Place - Air Handling Unit Rooftop Low Bay #38 - HVAC - 

In [8]:
equipment

2024-02-14 16:30:00    0.0
2024-02-14 16:45:00    0.0
2024-02-14 17:00:00    0.0
2024-02-14 17:15:00    0.0
2024-02-14 17:30:00    0.0
                      ... 
2024-04-30 19:00:00    0.0
2024-04-30 19:15:00    0.0
2024-04-30 19:30:00    0.0
2024-04-30 19:45:00    0.0
2024-04-30 20:00:00    0.0
Name: (9901 Linn Station Road, Training Room 815 Floor Box - Panel B8-1, Plug Load, America/New_York, 2024-02-14 21:28:07), Length: 7307, dtype: float64

## Format and Export

In [9]:
# convert start and end times to strings
formatted_df = blocks_df[['site',
                          'building_system',
                          'equipment',
                          'verified_date',
                          'length',
                          'start',
                          'end',
                          'max_usage_in_block',
                          'avg_usage_in_block',
                          'total_usage_in_block',
                          'avg_usage',
                          'max_usage',
                          'total_usage',
                          'block_percentage_of_use']]

formatted_df['start'] = formatted_df['start'].dt.strftime('%Y-%m-%d %H:%M:%S')
formatted_df['end'] = formatted_df['end'].dt.strftime('%Y-%m-%d %H:%M:%S')

# convert block length in minutes to days hours minutes
formatted_df['length'] = formatted_df['length'].apply(lambda x: str(dt.timedelta(minutes=x)))

# formatted df rounding
formatted_df = formatted_df.round(2)

# change column names
formatted_df = formatted_df.rename(columns={'site':'Site',
                                            'building_system':'Building System',
                                            'verified_date':'Metering Start',
                                            'equipment':'Equipment',
                                            'length':'Block Length',
                                            'start':'Block Start Time',
                                            'end':'Block End Time',
                                            'max_usage_in_block':'Max Usage in Block (kW)',
                                            'avg_usage_in_block':'Average Usage in Block (kW)',
                                            'total_usage_in_block':'Total Usage in Block (kWh)',
                                            'avg_usage':'Average Usage (kW)',
                                            'max_usage':'Max Usage (kW)',
                                            'total_usage':'Total Usage (kWh)',
                                            'block_percentage_of_use':'Block Percentage of Total Use (%)'
                                            })

portfolio_summary_df = pd.DataFrame(data={
                                    'Count of Unique Equipment': formatted_df.groupby('Site')['Equipment'].nunique(),
                                    'Metering Start': formatted_df.groupby('Site')['Metering Start'].max(),
                                    'Count of Instances': formatted_df.groupby('Site').count()['Building System'],
                                    'Total Usage in Blocks (kWh)': formatted_df.groupby('Site').sum()['Total Usage in Block (kWh)']})

# # export to different sheets in a workbook, one for each site
with pd.ExcelWriter('Honeywell/Honeywell_ContinuousUsage.xlsx') as writer:
    portfolio_summary_df.to_excel(writer, sheet_name='Site Summary')

    for site in formatted_df['Site'].unique():
        site_formatted_df = formatted_df[formatted_df['Site'] == site]

        site_summary_df = pd.DataFrame(data={
                                    'Building System': site_formatted_df.groupby('Equipment').apply(lambda x: x['Building System'].values[0]),
                                    'Metering Start': site_formatted_df.groupby('Equipment').apply(lambda x: x['Metering Start'].values[0]),
                                    'Count of Instances': site_formatted_df.groupby('Equipment').count()['Building System'],
                                    'Total Usage in Blocks (kWh)': site_formatted_df.groupby('Equipment').sum()['Total Usage in Block (kWh)'],
                                    'Total Usage (kWh)': site_formatted_df.groupby('Equipment').sum()['Total Usage (kWh)'],
                                    'Percentage of Total Use in Blocks (%)': (site_formatted_df.groupby('Equipment').sum()['Total Usage in Block (kWh)'] / site_formatted_df.groupby('Equipment').sum()['Total Usage (kWh)']),
                                    'Longest Block': site_formatted_df.groupby('Equipment').max()['Block Length']
                                    })

        site_summary_df.to_excel(writer, sheet_name=site, index=True)

    formatted_df.to_excel(writer, sheet_name='All Instances', index=True)

/var/folders/ld/px3lfl4j771_lns2jvpqcpfr0000gn/T/ipykernel_70128/862417038.py:47: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/ld/px3lfl4j771_lns2jvpqcpfr0000gn/T/ipykernel_70128/862417038.py:60: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/ld/px3lfl4j771_lns2jvpqcpfr0000gn/T/ipykernel_70128/862417038.py:61: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/ld/px3lfl4j771_lns2jvpqcpfr